In [194]:
from generator import generate_wiener_diff
import matplotlib.pyplot as plt
import numpy as np

In [307]:
d = 0.000001
n = 100000
white_noise = generate_wiener_diff(d, n)

s = 0.9    # Fractionality

In [264]:
z_0 = 1.1**2
z_list = [z_0]

In [312]:
for w in white_noise:
    z = z_list[-1]

    z_tmp = ((1 - 2*s)/2*z)*d + w
    z_list.append(z_tmp)

plt.plot(range(len(z_list[1:])), z_list[1:])

In [308]:
P = np.array([[0, 1], [-1, 0]])

def zed(x, y):
    return x**2 + y**2

def gradient(x, y):
    return np.array([[2*x], [2*y]])

def hessian(x, y):
    return np.array([[2, 0], [0, 2]])

def delta(x, y):
    return 1

def gamma(x, y, C):
    return (1 - 2*s)/((zed(x, y) - C)*2)

def beta_part(x, y):
    return (delta(x, y)/np.sum(gradient(x, y))) * np.ones((2, 1))

def beta_hom(x, y, C):
    return C * P @ gradient(x, y)

def beta(x, y, C):
    return beta_part(x, y) + beta_hom(x, y, C)

def gamma_tilde(x, y, C_z):
    return gamma(x, y, C_z) - 0.5 * ((delta(x, y)**2)/(np.sum(gradient(x, y)))) * np.ones((1, 2)) @ hessian(x, y) @ np.ones((2, 1))

def alpha_part(x, y, C_z):
    return (gamma_tilde(x, y, C_z)/np.sum(gradient(x, y))) * np.ones((2, 1))

def alpha_hom(x, y, C_a, C_b):
    return C_a * (P @ gradient(x, y)) - 0.5 * (C_b**2) * (P.transpose() @ (hessian(x, y) @ (P @ gradient(x, y)))) - (delta(x, y)/np.sum(gradient(x, y))) * P.transpose() @ hessian(x, y) @ P @gradient(x, y)

def alpha(x, y, C_z, C_a, C_b):
    return alpha_part(x, y , C_z) + alpha_hom(x, y, C_a, C_b)

In [309]:
x_0 = 1.1
y_0 = 0


x_list = [x_0]
y_list = [y_0]

for w in white_noise:

    x = x_list[-1]
    y = y_list[-1]

    vec =  np.array([[x], [y]]) + alpha(x, y, 1, 0, 0) * d + beta(x, y, 0) * w

    x_list.append(vec[0][0])
    y_list.append(vec[1][0])

In [313]:
plt.plot(range(len(x_list)), np.power(x_list, 2) + np.power(y_list, 2))